# Demo of _narrative graphs_


In [1]:
!rm output/demo_db.sqlite

In [2]:
from pandas import DataFrame
from sklearn.datasets import fetch_20newsgroups

n_docs = 2000

data_bunch = fetch_20newsgroups()
docs = data_bunch.data[:n_docs]
categories = [data_bunch.target_names[target] for target in data_bunch.target][:n_docs]

In [3]:
from narrativegraph import NarrativeGraph

model = NarrativeGraph(sqlite_db_path="output/demo_db.sqlite")
model.fit(docs, categories=categories)



INFO:narrativegraph.narrativegraph:Adding 2000 documents to database
Extracting triplets: 100%|██████████| 2000/2000 [00:41<00:00, 48.34it/s] 


In [ ]:
model.show_graph(max_nodes=100)

In [ ]:
model.get_docs(n=1000)